In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_AI_KEY")

### Overview

In [3]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [4]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
####INDEXING####

# Load Documents 
loader = WebBaseLoader(
  web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
  bs_kwargs=dict(
    parse_only=bs4.SoupStrainer(
      class_ = ("post-content", "post-title", "post-header")
    )

  ),

)

documents = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

# Embed 
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

#### Code explain
* **loader = WebBaseLoader(...)**:
  * This line creates an instance of WebBaseLoader and assigns it to the variable loader. The purpose of this loader is to fetch and process web content from specified URLs.

* **web_paths**=("https://lilianweng.github.io/posts/2023-06-23-agent/",):
  * This argument specifies the URL(s) from which the loader will fetch content. In this case, it’s a      single URL in a tuple. The comma after the URL indicates that it's a tuple, even though it contains only one item.

* **bs_kwargs=dict(...)**:
  * This parameter passes keyword arguments to a Beautiful Soup (bs4) parsing function. The bs_kwargs dictionary allows you to customize how the HTML content is parsed.

* **parse_only=bs4.SoupStrainer(...)**:
  * This specifies that only certain parts of the HTML document should be parsed. SoupStrainer is a class from Beautiful Soup that can filter out parts of the document based on specified criteria, improving performance by avoiding parsing unnecessary elements.

* **class_=("post-content", "post-title", "post-header")**:
  * This argument filters the parsed content to include only those elements with the class names "post-content", "post-title", or "post-header". This means that only these sections of the HTML will be extracted and processed, while the rest of the page will be ignored.


In [6]:
#### RETRIEVAL and GENERATION####

# Prompt
prompt = hub.pull("rlm/rag-prompt")


### Code explain
prompt = hub.pull("rlm/rag-prompt"):

* The **hub.pull** function is used to download or retrieve a resource by its name or identifier, "rlm/rag-prompt", from a remote hub.
* "rlm/rag-prompt" is the unique identifier for the resource or model that’s being fetched, likely a pre-trained model or a prompt configuration designed for Retrieval-Augmented Generation (RAG) tasks. 

In [7]:
#LLM model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

#A temperature of 0 makes the model’s responses more deterministic and focused, reducing the likelihood of random or creative variations. This setting is ideal for use cases where consistent, accurate responses are preferred over creative, diverse outputs.

In [8]:
# Post processing
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in documents)


In [9]:
# Chain
rag_chain = (
  {"context":retriever | format_docs, "question":RunnablePassthrough()}
  | prompt
  | llm 
  | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition involves breaking down large tasks into smaller, manageable subgoals to efficiently handle complex tasks. Techniques like Chain of Thought and Tree of Thoughts help models decompose hard tasks into simpler steps. Task decomposition can be done through simple prompting, task-specific instructions, or with human inputs.'

### Code explain
**{"context": retriever | format_docs, "question": RunnablePassthrough()}**:

* This dictionary acts as the initial step in the chain, specifying the input structure for the RAG process:

* "context": retriever | format_docs:
  This part retrieves and formats relevant documents for context.
  retriever is likely a component that fetches relevant documents or knowledge base entries based on the question.

* format_docs formats these documents into a form that’s usable by the prompt. The | operator chains these two steps, first retrieving and then formatting.

* "question": RunnablePassthrough():
  RunnablePassthrough() takes the question as-is without modifying it and passes it along in the pipeline.

* | prompt:
  This chains the result from the previous step into prompt, where the formatted context and question are combined to form a complete prompt. This prompt will then be passed to the language model for response generation.
* prompt (from the earlier snippet) is likely a pre-defined template or prompt configuration specific to the task.

* | llm:
  This step takes the generated prompt and feeds it to the language model (llm), which produces a response based on the context and question.
  llm was previously set up with ChatOpenAI configured to use gpt-3.5-turbo.

* | StrOutputParser():
  This final step parses the model’s output. StrOutputParser() likely processes the model’s response, converting it into a clean string format for easier interpretation or display.